# analyze the 100 sc items 

- we will look at the 100 datasets
- figure out why we dropped some 
- plot some stats


## conclusion: no idea how i generated the x40 list... so this notebook is shite

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

import matplotlib as mpl
mpl.rc("figure", dpi=200)

In [ ]:
%time 
# load the data
from natto import input 
from natto import process
from ubergauss import tools


names = input.get100names()

def load_and_score(dataset_name):
    data = input.load100(dataset_name,path = '/home/ubuntu/repos/natto/natto/data')
    gs= process.annotate_genescores(data, mingenes = 200)
    gs.X = np.expm1(gs.X)
    return gs 

annotated = tools.xmap(load_and_score,names)

#data = [input.load100(n,path = '/home/ubuntu/repos/natto/natto/data')  for n in names]
#annotated = [process.annotate_genescores(d, mingenes = 1000) for d in data]  

In [ ]:
# so we collect stats and put them in a table 



from matplotlib import pyplot as plt
import numpy as np
from lmz import *
import seaborn as sns 
import pandas as pd 
pd.options.display.max_rows = 1000

data = {'names': names}


# lets say we need 2k annotated cells
data['annotated cells']=[ sum(a.obs['true']!=-1) for a in annotated]
data['diverse cells'] = [ np.sum( np.array([ len(row.indices) for row in a.X]) > 1000 ) for a in annotated]
data['numgoodgenes'] = [ sum(a.varm['scores']!=np.NINF) for a in annotated]
data['reads per cell'] = [ np.mean(np.sum(a.X,axis= 1)  ) for a in annotated]


table = pd.DataFrame(data)
table

In [ ]:
annotated[0].X[0].data

In [ ]:
if False:
    # another problem was that many cells could have too few reads 
    for name, ann in zip(names, annotated):
        print(name)
        plt.hist(np.sum(ann.X, axis = 1), bins = 75)
        plt.show()

In [ ]:
col = Zip(names, annot)
col = [c for c in col if c[1]>1500 ]
print ([c[0][:5] for c in col])
print(len(col))

# now make sure that 

In [ ]:
# so now we have some tissues with few representatives, lets clean this also


col = [c[0] for c in col] # list of names
a,b = np.unique([c[:5] for c in col ], return_counts=True)
itemcounts = Zip(a,b)
shitlist =  [item for item, count in itemcounts if count < 4]
flist = [c for c in col if c[:5] not in shitlist]


In [ ]:
flist

In [ ]:
import seaborn as sns 
import pandas as pd 

pdata = pd.DataFrame({'name':a,'count':b})
sns.barplot(data  = pdata,x='name',y='count')

In [ ]:

data['reads > 1400'] = [ sum(np.sum(a.X,axis= 1) > 1500 ) for a in annotated]
